# 2022-02-21 • FENS 2022 Abstract

As usually, the reusable code of the previous notebook 
has been added in [`pkg/VoltageToMap/src/`](https://github.com/tfiers/voltage-to-wiring-sim/tree/main/pkg/VoltageToMap/src),
and is imported below.

## Setup

In [1]:
# 

In [2]:
using Revise  # Reloads src code without having to restart kernel

In [3]:
using MyToolbox

In [5]:
using VoltageToMap

## Params & sim

Short warm-up run. Get compilation out of the way.

In [17]:
p0 = params
@set p0.sim.inputs   = previous_N_30_inputs
@set p0.sim.duration = 1 * minutes;

In [16]:
@time sim(p0.sim);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
  4.219216 seconds (4.74 M allocations: 361.029 MiB, 2.70% gc time, 43.22% compilation time)


In [18]:
p = params
@set p.sim.inputs   = realistic_N_6600_inputs
@set p.sim.duration = 10 * minutes
@set p.sim.synapses.Δg_multiplier = 0.06
dumpc(p)

ExperimentParams
  seed: 22022022
  sim: SimParams
    duration: 10.0
    Δt: 0.0001
    seed: 22022022
    inputs: PoissonInputsParams
      N_unconn: 100
      N_exc: 5200
      N_inh: 1300
      N_conn: 6500
      N: 6600
      spike_rates: LogNormal
        μ: 1.08629
        σ: 0.774597
    synapses: SynapseParams
      Δg_exc: 4.0e-10
      Δg_inh: 1.6e-9
      Δg_multiplier: 1.0
      E_exc: 0.0
      E_inh: -0.065
      g_t0: 0.0
      τ: 0.007
    izh_neuron: IzhikevichParams
      C: 1.0e-10
      k: 7.0e-7
      v_rest: -0.06
      v_thr: -0.04
      a: 30.0
      b: -2.0e-9
      v_peak: 0.035
      v_reset: -0.05
      Δu: 1.0e-10
      v_t0: -0.06
      u_t0: 0.0
    imaging: VoltageImagingParams
      spike_SNR: 20.0
      spike_height: 0.095
      σ_noise: 0.00475
    num_timesteps: 100000
  conntest: ConnTestParams
    STA_window_length: 0.1
    num_shuffles: 100
    seed: 22022022
  evaluation: EvaluationParams
    num_tested_neurons_per_group: 40
    seed: 22022022


In [8]:
t, v, input_spikes = @time sim(p);

Progress: 100%|█████████████████████████████████████████| Time: 0:02:51mmm9mmmm9mmmmm


173.195922 seconds (24.14 M allocations: 2.302 GiB, 0.28% gc time, 1.40% compilation time)


In [9]:
num_spikes = length.(input_spikes)

ComponentVector{Int64}(conn = (exc = [931, 526, 658, 1256, 1091, 381, 622, 356, 877, 854  …  1133, 621, 575, 861, 717, 654, 887, 1198, 849, 895], inh = [887, 898, 816, 637, 1227, 1076, 1096, 634, 538, 518  …  569, 1182, 721, 882, 1271, 984, 793, 769, 333, 1259]), unconn = [1404, 881, 992, 1227, 951, 632, 1242, 1004, 810, 716  …  518, 309, 1253, 617, 1221, 646, 1502, 1047, 693, 586])

## Plot

In [14]:
import PyPlot
using Sciplotlib

In [15]:
""" tzoom = [200ms, 600ms] e.g. """
function plotsig(t, sig, tzoom = nothing; ax = nothing, clip_on=false, kw...)
    isnothing(tzoom) && (tzoom = t[[1, end]])
    izoom = first(tzoom) .≤ t .≤ last(tzoom)
    if isnothing(ax)
        plot(t[izoom], sig[izoom]; clip_on, kw...)
    else
        plot(t[izoom], sig[izoom], ax; clip_on, kw...)
    end
end;

In [ ]:
plotsig(t, v/mV, [0s, 4seconds], xlabel="Time (s)", hylabel="mV");

## Imaging noise

In [20]:
izh_params = cortical_RS

imaging_spike_SNR  #=::Float64=#  = 20
spike_height       #=::Float64=#  = izh_params.v_peak - izh_params.vr
σ_noise            #=::Float64=#  = spike_height / imaging_spike_SNR;

In [21]:
noise = randn(length(v)) * σ_noise
vimsig = v + noise;

In [ ]:
ax = plotsig(t, vimsig / mV, [200ms,1200ms], xlabel="Time (s)", hylabel="mV", alpha=0.7);
plotsig(t, v / mV, [200ms,1200ms], xlabel="Time (s)", hylabel="mV"; ax);

## Window

In [23]:
window_duration    #=::Float64=#  = 100 * ms;

In [24]:
const Δt = p.Δt
const win_size = round(Int, window_duration / Δt)
const t_win = linspace(zero(window_duration), window_duration, win_size)

function calc_STA(presynaptic_spikes)
    STA = zeros(eltype(vimsig), win_size)
    win_starts = round.(Int, presynaptic_spikes / Δt)
    num_wins = 0
    for a in win_starts
        b = a + win_size - 1
        if b ≤ lastindex(vimsig)
            STA .+= @view vimsig[a:b]
            num_wins += 1
        end
    end
    STA ./= num_wins
    return STA
end;

In [25]:
function plotSTA(presynspikes)
    STA = calc_STA(presynspikes)
    plot(t_win/ms, STA/mV)
end;

In [ ]:
presynspikes = input_spikes.conn.exc[44]
plotSTA(presynspikes);

## Test connection

In [27]:
num_shuffles       #=::Int    =#  = 100;

In [28]:
to_ISIs(spiketimes) = [first(spiketimes); diff(spiketimes)]  # copying
to_spiketimes!(ISIs) = cumsum!(ISIs, ISIs)                   # in place

(presynspikes |> to_ISIs |> to_spiketimes!) ≈ presynspikes   # test

true

In [29]:
shuffle_ISIs(spiketimes) = to_spiketimes!(shuffle!(to_ISIs(spiketimes)));

In [30]:
test_statistic(spiketimes) = spiketimes |> calc_STA |> mean;

Note difference with 2021: there it was peak-to-peak (max - min). Here it is mean.

In [31]:
function test_connection(presynspikes)
    real_t = test_statistic(presynspikes)
    shuffled_t = Vector{typeof(real_t)}(undef, num_shuffles)
    for i in eachindex(shuffled_t)
        shuffled_t[i] = test_statistic(shuffle_ISIs(presynspikes))
    end
    N_shuffled_larger = count(shuffled_t .> real_t)
    return if N_shuffled_larger == 0
        p_value = 1 / num_shuffles
    else
        p_value = N_shuffled_larger / num_shuffles
    end
end;

## Results

In [34]:
resetrng!(20220222);

In [35]:
num_trains = 40
println("Average p(shuffled trains with higher STA mean).")
println("(N = $(num_trains) input spike trains per category)")

p_exc    = Float64[]
p_inh    = Float64[]
p_unconn = Float64[]

for (groupname, spiketrains, pvals) in (
        ("excitatory",    input_spikes.conn.exc, p_exc),
        ("inhibitory",    input_spikes.conn.inh, p_inh),
        ("unconnected",   input_spikes.unconn, p_unconn),
    )
    for spiketrain in spiketrains[1:num_trains]
        push!(pvals, test_connection(spiketrain))
        print("."); flush(stdout)
    end
    @printf "%12s: %.3g\n" groupname mean(pvals)
end

Average p(shuffled trains with higher mean).
(N = 40 input spike trains per category)
........................................  excitatory: 0.416
........................................  inhibitory: 0.716
........................................ unconnected: 0.508


In [ ]:
fig, ax = plt.subplots(figsize=(3.4,3))
function plotdot(y, x, c, jitter=0.28)
    N = length(y)
    x -= 0.35
    plot(x*ones(N) + (rand(N).-0.5)*jitter, y, "o", color=c, ms=4.2, markerfacecolor="none", clip_on=false)
    plot(x+0.35, mean(y), "k.", ms=10)
end
plotdot(p_exc,    1, "C2"); ax.text(1-0.16, -0.1, "excitatory"; color="C2", ha="center")
plotdot(p_unconn, 2, "C0"); ax.text(2-0.16, -0.1, "unconnected"; color="C0", ha="center")
plotdot(p_inh,    3, "C1"); ax.text(3-0.16, -0.1, "inhibitory"; color="C1", ha="center")
ax.boxplot([p_exc, p_unconn, p_inh], widths=0.2, medianprops=Dict("color"=>"black"))
set(ax, xlim=(0.33, 3.3), ylim=(0, 1), xaxis=:off)
hylabel(ax, L"p(\, \mathrm{shuffled\ \overline{STA}} \ > \ \mathrm{actual\ \overline{STA}}\, )"; dy=10);

Proportion of shuffled spike trains for which `mean(STA)` is higher than the unshuffled spike train.

Excitatory (green), unconnected (blue), and inhibitory (orange) input neurons.


10-minute simulation with a total of 6500 connected input neurons.